In [12]:
import cv2
import os
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import math
from scipy import signal
import re
from sklearn.metrics import mean_absolute_error
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
import glob



In [13]:

def video_to_frames(sub,activity,cam,trial):

    output_folder="D:\ML\FYP\Dataset\\New_Dataset\\New\\"+sub+"\Outputs\\frames\\"
    video_path="D:\ML\FYP\Dataset\\New_Dataset\\New\\"+sub+"\\"+sub+"_CameraData_Orig\\"+sub+"_TrialVideos_Orig\\"+sub+"_"+activity+"_TV_Orig\\"+sub+"_"+activity+"_Cam"+cam+"_Orig_Trial"+trial+".mp4"
    vidObj = cv2.VideoCapture(video_path)
    count=0
    success = True
    while success:

        success, image = vidObj.read()
        
        if image is not None:
            cv2.imwrite(output_folder+"frame%d.jpg" % count, image)
            print(count)
            count += 1
    return count

    

In [14]:
def original_time(sub,activity,count,cam,trial_num):
    csv_file=pd.read_csv("D:\ML\FYP\Dataset\\New_Dataset\\New\\"+sub+"\\"+sub+"_FramesData_Orig\\"+sub+"_"+activity+"_FD_Orig\\"+sub+"_"+activity+"_Orig_Trial"+trial_num+".csv")
    org=list(csv_file['Camera'])
    c=org.index(int(cam))
    org=list(csv_file['TimeStamp'])[c:c+count]
    print(org)
    return org

In [15]:
   
def xyz(sub,activity,count,cam,trial_num):
    import cv2
    import mediapipe as mp
    import pandas as pd
    import glob
    import re
    import numpy as np
    import matplotlib.pyplot as plt

    path="D:\ML\FYP\Dataset\\New_Dataset\\New\\"+sub+"\\Outputs\\frames"
    pdf_name="D:\ML\FYP\Dataset\\New_Dataset\\New\\"+sub+"\\Outputs\\xyz\\"+sub+"_Cam"+cam+"_T"+trial_num+"_xyz.csv"
    mp_pose = mp.solutions.pose

    pose_image = mp_pose.Pose(static_image_mode=False,min_detection_confidence=0.5)

    mp_drawing = mp.solutions.drawing_utils

    def detectPose(image_pose, pose, draw=False, display=False):
        
        original_image = image_pose.copy()
        
        image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
        
        resultant = pose.process(image_in_RGB)

        if resultant.pose_landmarks and draw:    

            mp_drawing.draw_landmarks(image=original_image, landmark_list=resultant.pose_landmarks,
                                    connections=mp_pose.POSE_CONNECTIONS,
                                    landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                                thickness=3, circle_radius=3),
                                    connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
                                                                                thickness=2, circle_radius=2))
        
    
        try:
            landmarks = resultant.pose_world_landmarks.landmark
            return landmarks
            
        except:
            print('no landmarks detected')
            pass

    landmarks_df=pd.DataFrame(columns=['Frame_Num','Time','LEFT_SHOULDER_X','LEFT_SHOULDER_Y','LEFT_SHOULDER_Z',
                                                                                                    'RIGHT_SHOULDER_X','RIGHT_SHOULDER_Y','RIGHT_SHOULDER_Z',
                                                                                                    'LEFT_ELBOW_X','LEFT_ELBOW_Y','LEFT_ELBOW_Z',
                                                                                                    'RIGHT_ELBOW_X','RIGHT_ELBOW_Y','RIGHT_ELBOW_Z',
                                                                                                    'LEFT_WRIST_X','LEFT_WRIST_Y','LEFT_WRIST_Z',
                                                                                                    'RIGHT_WRIST_X','RIGHT_WRIST_Y','RIGHT_WRIST_Z',
                                                                                                    'LEFT_PINKY_X','LEFT_PINKY_Y','LEFT_PINKY_Z',
                                                                                                    'RIGHT_PINKY_X','RIGHT_PINKY_Y','RIGHT_PINKY_Z',
                                                                                                    'LEFT_INDEX_X','LEFT_INDEX_Y','LEFT_INDEX_Z',
                                                                                                    'RIGHT_INDEX_X','RIGHT_INDEX_Y','RIGHT_INDEX_Z',
                                                                                                    'LEFT_THUMB_X','LEFT_THUMB_Y','LEFT_THUMB_Z',
                                                                                                    'RIGHT_THUMB_X','RIGHT_THUMB_Y','RIGHT_THUMB_Z',
                                                                                                    'LEFT_HIP_X','LEFT_HIP_Y','LEFT_HIP_Z',
                                                                                                    'RIGHT_HIP_X','RIGHT_HIP_Y','RIGHT_HIP_Z',
                                                                                                    'LEFT_KNEE_X','LEFT_KNEE_Y','LEFT_KNEE_Z',
                                                                                                    'RIGHT_KNEE_X','RIGHT_KNEE_Y','RIGHT_KNEE_Z',
                                                                                                    'LEFT_ANKLE_X','LEFT_ANKLE_Y','LEFT_ANKLE_Z',
                                                                                                    'RIGHT_ANKLE_X','RIGHT_ANKLE_Y','RIGHT_ANKLE_Z',
                                                                                                    'LEFT_HEEL_X','LEFT_HEEL_Y','LEFT_HEEL_Z',
                                                                                                    'RIGHT_HEEL_X','RIGHT_HEEL_Y','RIGHT_HEEL_Z',
                                                                                                    'LEFT_FOOT_INDEX_X','LEFT_FOOT_INDEX_Y','LEFT_FOOT_INDEX_Z',
                                                                                                    'RIGHT_FOOT_INDEX_X','RIGHT_FOOT_INDEX_Y','RIGHT_FOOT_INDEX_Z'])
    # Here we will read our image from the specified path to detect the pose
    print(landmarks_df)
    image_path = glob.glob(path+"/*.jpg")


    for file in image_path:
        temp = re.findall(r'\d+', file)
        frame_num = list(map(int, temp))
        # frame_num=[frame_num[2]]
        frame_num=frame_num[1]
        print(frame_num)                 #change the list index according to the number of numeric char in your file path
        print("********************************************")
        output = cv2.imread(file)
        landmarks=detectPose(output, pose_image, draw=True, display=True)
        try:

            LEFT_SHOULDER_X,LEFT_SHOULDER_Y,LEFT_SHOULDER_Z = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].z
            RIGHT_SHOULDER_X,RIGHT_SHOULDER_Y,RIGHT_SHOULDER_Z = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].z

            LEFT_ELBOW_X,LEFT_ELBOW_Y,LEFT_ELBOW_Z = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].z

            RIGHT_ELBOW_X,RIGHT_ELBOW_Y,RIGHT_ELBOW_Z = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].z

            LEFT_WRIST_X,LEFT_WRIST_Y,LEFT_WRIST_Z = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y,landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].z

            RIGHT_WRIST_X,RIGHT_WRIST_Y,RIGHT_WRIST_Z = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].z

            LEFT_PINKY_X,LEFT_PINKY_Y,LEFT_PINKY_Z = landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].x,landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].y,landmarks[mp_pose.PoseLandmark.LEFT_PINKY.value].z

            RIGHT_PINKY_X,RIGHT_PINKY_Y,RIGHT_PINKY_Z = landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_PINKY.value].z

            LEFT_INDEX_X,LEFT_INDEX_Y,LEFT_INDEX_Z = landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].y,landmarks[mp_pose.PoseLandmark.LEFT_INDEX.value].z

            RIGHT_INDEX_X,RIGHT_INDEX_Y,RIGHT_INDEX_Z = landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_INDEX.value].z

            LEFT_THUMB_X,LEFT_THUMB_Y,LEFT_THUMB_Z = landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].x,landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].y,landmarks[mp_pose.PoseLandmark.LEFT_THUMB.value].z

            RIGHT_THUMB_X,RIGHT_THUMB_Y,RIGHT_THUMB_Z = landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_THUMB.value].z

            LEFT_HIP_X,LEFT_HIP_Y,LEFT_HIP_Z = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].z

            RIGHT_HIP_X,RIGHT_HIP_Y,RIGHT_HIP_Z = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].z

            LEFT_KNEE_X,LEFT_KNEE_Y,LEFT_KNEE_Z = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].z

            RIGHT_KNEE_X,RIGHT_KNEE_Y,RIGHT_KNEE_Z = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].z

            LEFT_ANKLE_X,LEFT_ANKLE_Y,LEFT_ANKLE_Z = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].z

            RIGHT_ANKLE_X,RIGHT_ANKLE_Y,RIGHT_ANKLE_Z = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].z

            LEFT_HEEL_X,LEFT_HEEL_Y,LEFT_HEEL_Z = landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].y,landmarks[mp_pose.PoseLandmark.LEFT_HEEL.value].z
            RIGHT_HEEL_X,RIGHT_HEEL_Y,RIGHT_HEEL_Z = landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_HEEL.value].z

            LEFT_FOOT_INDEX_X,LEFT_FOOT_INDEX_Y,LEFT_FOOT_INDEX_Z = landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].y,landmarks[mp_pose.PoseLandmark.LEFT_FOOT_INDEX.value].z

            RIGHT_FOOT_INDEX_X,RIGHT_FOOT_INDEX_Y,RIGHT_FOOT_INDEX_Z = landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].y,landmarks[mp_pose.PoseLandmark.RIGHT_FOOT_INDEX.value].z
            
            
            
            
            landmarks_df.loc[len(landmarks_df)]= [frame_num,0,LEFT_SHOULDER_X,LEFT_SHOULDER_Y,LEFT_SHOULDER_Z,
                                RIGHT_SHOULDER_X,RIGHT_SHOULDER_Y,RIGHT_SHOULDER_Z,
                                LEFT_ELBOW_X,LEFT_ELBOW_Y,LEFT_ELBOW_Z,
                                RIGHT_ELBOW_X,RIGHT_ELBOW_Y,RIGHT_ELBOW_Z,
                                LEFT_WRIST_X,LEFT_WRIST_Y,LEFT_WRIST_Z,
                                RIGHT_WRIST_X,RIGHT_WRIST_Y,RIGHT_WRIST_Z ,
                                LEFT_PINKY_X,LEFT_PINKY_Y,LEFT_PINKY_Z, 
                                RIGHT_PINKY_X,RIGHT_PINKY_Y,RIGHT_PINKY_Z, 
                                LEFT_INDEX_X,LEFT_INDEX_Y,LEFT_INDEX_Z,
                                RIGHT_INDEX_X,RIGHT_INDEX_Y,RIGHT_INDEX_Z,
                                LEFT_THUMB_X,LEFT_THUMB_Y,LEFT_THUMB_Z, 
                                RIGHT_THUMB_X,RIGHT_THUMB_Y,RIGHT_THUMB_Z , 
                                LEFT_HIP_X,LEFT_HIP_Y,LEFT_HIP_Z, 
                                RIGHT_HIP_X,RIGHT_HIP_Y,RIGHT_HIP_Z,
                                LEFT_KNEE_X,LEFT_KNEE_Y,LEFT_KNEE_Z ,
                                RIGHT_KNEE_X,RIGHT_KNEE_Y,RIGHT_KNEE_Z, 
                                LEFT_ANKLE_X,LEFT_ANKLE_Y,LEFT_ANKLE_Z, 
                                RIGHT_ANKLE_X,RIGHT_ANKLE_Y,RIGHT_ANKLE_Z, 
                                LEFT_HEEL_X,LEFT_HEEL_Y,LEFT_HEEL_Z,
                                RIGHT_HEEL_X,RIGHT_HEEL_Y,RIGHT_HEEL_Z,
                                LEFT_FOOT_INDEX_X,LEFT_FOOT_INDEX_Y,LEFT_FOOT_INDEX_Z,
                                RIGHT_FOOT_INDEX_X,RIGHT_FOOT_INDEX_Y,RIGHT_FOOT_INDEX_Z]

            print('success')                                                                                            
        except:
            print('failure')
            if len(landmarks_df)>0:
                landmarks_df.loc[len(landmarks_df)]=landmarks_df.loc[len(landmarks_df)-1]
            else:
                l=[frame_num,0]
                l.extend([0 for x in range(66)])
                landmarks_df.loc[len(landmarks_df)]=l

    landmarks_df= landmarks_df.sort_values(by=['Frame_Num'])
    print("length of df:" + str(len(landmarks_df)))
    org=original_time(sub,activity,count,cam,trial_num)
    landmarks_df['Time']=[x/1000 for x in org]
    landmarks_df.to_csv(pdf_name, index=False)


In [16]:
def mc_angles(sub,activity,trial):


    
    
    mc_columns=["Time","Frame","LED_TRIGGER","L.AnkleLateral_X","L.AnkleLateral_Y",	"L.AnkleLateral_Z",
            "L.AnkleMedial_X","L.AnkleMedial_Y","L.AnkleMedial_Z" ,        
            "L.FootHeel_X",	"L.FootHeel_Y",	"L.FootHeel_Z",	
            "L.KneeMedial_X","L.KneeMedial_Y","L.KneeMedial_Z",
            "L.KneeLateral_X",	"L.KneeLateral_Y",	"L.KneeLateral_Z",	
            "L.LowerArmLateral_X",	"L.LowerArmLateral_Y",	"L.LowerArmLateral_Z",
            "L.LowerArmMedial_X","L.LowerArmMedial_Y","L.LowerArmMedial_Z",
            "L.ShoulderFront_X","L.ShoulderFront_Y","L.ShoulderFront_Z",
            "L.ShoulderRear_X","L.ShoulderRear_Y","L.ShoulderRear_Z",
            "L.ToeMedial_X","L.ToeMedial_Y","L.ToeMedial_Z",	
            "LASIS_X",	"LASIS_Y",	"LASIS_Z",	"L.WristMedial_X", "L.WristMedial_Y","L.WristMedial_Z",
            "L.WristLateral_X","L.WristLateral_Y","L.WristLateral_Z",
            "R.AnkleLateral_X",	"R.AnkleLateral_Y",	"R.AnkleLateral_Z",	
            "R.AnkleMedial_X","R.AnkleMedial_Y","R.AnkleMedial_Z" ,
            "R.FootHeel_X",	"R.FootHeel_Y",	"R.FootHeel_Z",	
            "R.KneeMedial_X","R.KneeMedial_Y","R.KneeMedial_Z",
            "R.KneeLateral_X",	"R.KneeLateral_Y",	"R.KneeLateral_Z",	
            "R.LowerArmLateral_X",	"R.LowerArmLateral_Y",	"R.LowerArmLateral_Z",
            "R.LowerArmMedial_X","R.LowerArmMedial_Y","R.LowerArmMedial_Z",
            "R.ShoulderFront_X","R.ShoulderFront_Y","R.ShoulderFront_Z",
            "R.ShoulderRear_X","R.ShoulderRear_Y","R.ShoulderRear_Z",
            "R.ToeMedial_X","R.ToeMedial_Y","R.ToeMedial_Z",
            "RASIS_X",	"RASIS_Y",	"RASIS_Z", "R.WristMedial_X", "R.WristMedial_Y","R.WristMedial_Z",
            "R.WristLateral_X","R.WristLateral_Y","R.WristLateral_Z",]

    seg_col_name=['LEFT_ANKLE_HEEL', 'LEFT_ANKLE_FOOT_INDEX', 'LEFT_HEEL_FOOT_INDEX', 'LEFT_ANKLE_KNEE', 
                'LEFT_KNEE_HIP', 'LEFT_HIP_SHOULDER', 'LEFT_SHOULDER_ELBOW', 'LEFT_ELBOW_WRIST',
                'RIGHT_ANKLE_HEEL', 'RIGHT_ANKLE_FOOT_INDEX', 'RIGHT_HEEL_FOOT_INDEX', 'RIGHT_ANKLE_KNEE', 
                'RIGHT_KNEE_HIP', 'RIGHT_HIP_SHOULDER', 'RIGHT_SHOULDER_ELBOW', 'RIGHT_HIP_LEFT', 
                'RIGHT_SHOULDER_LEFT','RIGHT_ELBOW_WRIST']

    mae_df=pd.DataFrame(columns=seg_col_name)

   
    mc=pd.read_csv("D:\ML\FYP\Dataset\\New_Dataset\\New\\"+sub+"\Activity_Trials\\"+sub+"_"+activity+"_Trial"+trial+".csv")
    mc=pd.DataFrame(mc,columns=mc_columns)
    
    
    #KNEE
    mc["L.KneeLateral_Z"]=(mc["L.KneeLateral_Z"]+mc["L.KneeMedial_Z"])/2
    mc["L.KneeLateral_X"]=(mc["L.KneeLateral_X"]+mc["L.KneeMedial_X"])/2
    mc["L.KneeLateral_Y"]=(mc["L.KneeLateral_Y"]+mc["L.KneeMedial_Y"])/2
        
    mc["R.KneeLateral_Z"]=(mc["R.KneeLateral_Z"]+mc["R.KneeMedial_Z"])/2
    mc["R.KneeLateral_X"]=(mc["R.KneeLateral_X"]+mc["R.KneeMedial_X"])/2
    mc["R.KneeLateral_Y"]=(mc["R.KneeLateral_Y"]+mc["R.KneeMedial_Y"])/2

    #wrist
    mc["L.WristLateral_Z"]=(mc["L.WristLateral_Z"]+mc["L.WristMedial_Z"])/2
    mc["L.WristLateral_X"]=(mc["L.WristLateral_X"]+mc["L.WristMedial_X"])/2
    mc["L.WristLateral_Y"]=(mc["L.WristLateral_Y"]+mc["L.WristMedial_Y"])/2
    
    mc["R.WristLateral_Z"]=(mc["R.WristLateral_Z"]+mc["R.WristMedial_Z"])/2
    mc["R.WristLateral_X"]=(mc["R.WristLateral_X"]+mc["R.WristMedial_X"])/2
    mc["R.WristLateral_Y"]=(mc["R.WristLateral_Y"]+mc["R.WristMedial_Y"])/2
    
    
    #shoulder
    mc["L.ShoulderFront_Z"]=(mc["L.ShoulderFront_Z"]+mc["L.ShoulderRear_Z"])/2
    mc["L.ShoulderFront_X"]=(mc["L.ShoulderFront_X"]+mc["L.ShoulderRear_X"])/2
    mc["L.ShoulderFront_Y"]=(mc["L.ShoulderFront_Y"]+mc["L.ShoulderRear_Y"])/2
        
    mc["R.ShoulderFront_Z"]=(mc["R.ShoulderFront_Z"]+mc["R.ShoulderRear_Z"])/2
    mc["R.ShoulderFront_X"]=(mc["R.ShoulderFront_X"]+mc["R.ShoulderRear_X"])/2
    mc["R.ShoulderFront_Y"]=(mc["R.ShoulderFront_Y"]+mc["R.ShoulderRear_Y"])/2
    
    
    #elbow
    mc["L.LowerArmLateral_Z"]=(mc["L.LowerArmLateral_Z"]+mc["L.LowerArmMedial_Z"])/2
    mc["L.LowerArmLateral_X"]=(mc["L.LowerArmLateral_X"]+mc["L.LowerArmMedial_X"])/2
    mc["L.LowerArmLateral_Y"]=(mc["L.LowerArmLateral_Y"]+mc["L.LowerArmMedial_Y"])/2
    
    mc["R.LowerArmLateral_Z"]=(mc["R.LowerArmLateral_Z"]+mc["R.LowerArmMedial_Z"])/2
    mc["R.LowerArmLateral_X"]=(mc["R.LowerArmLateral_X"]+mc["R.LowerArmMedial_X"])/2
    mc["R.LowerArmLateral_Y"]=(mc["R.LowerArmLateral_Y"]+mc["R.LowerArmMedial_Y"])/2
        
        
        #ankle
    mc["L.AnkleLateral_Z"]=(mc["L.AnkleLateral_Z"]+mc["L.AnkleMedial_Z"])/2
    mc["L.AnkleLateral_Y"]=(mc["L.AnkleLateral_Y"]+mc["L.AnkleMedial_Y"])/2
    mc["L.AnkleLateral_X"]=(mc["L.AnkleLateral_X"]+mc["L.AnkleMedial_X"])/2
    
    mc["R.AnkleLateral_Z"]=(mc["R.AnkleLateral_Z"]+mc["R.AnkleMedial_Z"])/2
    mc["R.AnkleLateral_Y"]=(mc["R.AnkleLateral_Y"]+mc["R.AnkleMedial_Y"])/2
    mc["R.AnkleLateral_X"]=(mc["R.AnkleLateral_X"]+mc["R.AnkleMedial_X"])/2
    
    
    drop_columns=["L.LowerArmMedial_X","L.LowerArmMedial_Y","L.LowerArmMedial_Z","R.LowerArmMedial_X","R.LowerArmMedial_Y",
                    "R.LowerArmMedial_Z","L.ShoulderRear_X","L.ShoulderRear_Y","L.ShoulderRear_Z","R.ShoulderRear_X","R.ShoulderRear_Y",
                    "R.ShoulderRear_Z","L.KneeMedial_X","L.KneeMedial_Y","L.KneeMedial_Z","R.KneeMedial_X","R.KneeMedial_Y",
                    "R.KneeMedial_Z","L.AnkleMedial_Z","L.AnkleMedial_Y","L.AnkleMedial_X","R.AnkleMedial_Z","R.AnkleMedial_Y","R.AnkleMedial_X",
                    "L.WristMedial_X", "L.WristMedial_Y","L.WristMedial_Z", "R.WristMedial_X", "R.WristMedial_Y","R.WristMedial_Z"]
    
    for col in drop_columns:
        del mc[col]
    
    mc.columns =["Time","Frame_Num","ON_LED","LEFT_ANKLE_X","LEFT_ANKLE_Y","LEFT_ANKLE_Z","LEFT_HEEL_X","LEFT_HEEL_Y","LEFT_HEEL_Z","LEFT_KNEE_X",	
    "LEFT_KNEE_Y",	"LEFT_KNEE_Z",	"LEFT_ELBOW_X",	"LEFT_ELBOW_Y",	"LEFT_ELBOW_Z",	"LEFT_SHOULDER_X",	"LEFT_SHOULDER_Y",	
    "LEFT_SHOULDER_Z","LEFT_FOOT_INDEX_X","LEFT_FOOT_INDEX_Y","LEFT_FOOT_INDEX_Z","LEFT_HIP_X","LEFT_HIP_Y",	
    "LEFT_HIP_Z","LEFT_WRIST_X","LEFT_WRIST_Y","LEFT_WRIST_Z","RIGHT_ANKLE_X","RIGHT_ANKLE_Y","RIGHT_ANKLE_Z","RIGHT_HEEL_X",	"RIGHT_HEEL_Y",	"RIGHT_HEEL_Z",	
    "RIGHT_KNEE_X",	"RIGHT_KNEE_Y",	"RIGHT_KNEE_Z",	"RIGHT_ELBOW_X","RIGHT_ELBOW_Y","RIGHT_ELBOW_Z","RIGHT_SHOULDER_X",	
    "RIGHT_SHOULDER_Y",	"RIGHT_SHOULDER_Z",	"RIGHT_FOOT_INDEX_X","RIGHT_FOOT_INDEX_Y","RIGHT_FOOT_INDEX_Z","RIGHT_HIP_X",	
    "RIGHT_HIP_Y",	"RIGHT_HIP_Z","RIGHT_WRIST_X","RIGHT_WRIST_Y","RIGHT_WRIST_Z"]
        

    left_ankle= mc[['LEFT_ANKLE_X', 'LEFT_ANKLE_Y','LEFT_ANKLE_Z']].values.tolist()
    left_heel=mc[['LEFT_HEEL_X', 'LEFT_HEEL_Y','LEFT_HEEL_Z']].values.tolist()
    left_foot_index=mc[['LEFT_FOOT_INDEX_X', 'LEFT_FOOT_INDEX_Y','LEFT_FOOT_INDEX_Z']].values.tolist()
    left_knee=mc[['LEFT_KNEE_X', 'LEFT_KNEE_Y','LEFT_KNEE_Z']].values.tolist()
    left_hip=mc[['LEFT_HIP_X', 'LEFT_HIP_Y','LEFT_HIP_Z']].values.tolist()
    left_shoulder=mc[['LEFT_SHOULDER_X', 'LEFT_SHOULDER_Y','LEFT_SHOULDER_Z']].values.tolist()
    left_elbow=mc[['LEFT_ELBOW_X', 'LEFT_ELBOW_Y','LEFT_ELBOW_Z']].values.tolist()
    left_wrist=mc[['LEFT_WRIST_X', 'LEFT_WRIST_Y','LEFT_WRIST_Z']].values.tolist()
    right_ankle= mc[['RIGHT_ANKLE_X', 'RIGHT_ANKLE_Y','RIGHT_ANKLE_Z']].values.tolist()
    right_heel=mc[['RIGHT_HEEL_X', 'RIGHT_HEEL_Y','RIGHT_HEEL_Z']].values.tolist()
    right_foot_index=mc[['RIGHT_FOOT_INDEX_X', 'RIGHT_FOOT_INDEX_Y','RIGHT_FOOT_INDEX_Z']].values.tolist()
    right_knee=mc[['RIGHT_KNEE_X', 'RIGHT_KNEE_Y','RIGHT_KNEE_Z']].values.tolist()
    right_hip=mc[['RIGHT_HIP_X', 'RIGHT_HIP_Y','RIGHT_HIP_Z']].values.tolist()
    right_shoulder=mc[['RIGHT_SHOULDER_X', 'RIGHT_SHOULDER_Y','RIGHT_SHOULDER_Z']].values.tolist()
    right_elbow=mc[['RIGHT_ELBOW_X', 'RIGHT_ELBOW_Y','RIGHT_ELBOW_Z']].values.tolist()
    right_wrist=mc[['RIGHT_WRIST_X', 'RIGHT_WRIST_Y','RIGHT_WRIST_Z']].values.tolist()
        
    seg_df=pd.DataFrame()
    
    
    left_foot_ankle_knee,right_foot_ankle_knee,right_ankle_knee_hip,left_ankle_knee_hip,right_knee_hip_shoulder,left_knee_hip_shoulder,right_hip_shoulder_elbow,left_hip_shoulder_elbow,right_shoulder_elbow_wrist,left_shoulder_elbow_wrist= ([] for i in range(10))
    
    def find_angle(l1,l2,l3):
        origin = np.array(l2)

        vector1 = np.array(l1)
        vector2 = np.array(l3)

        relative_vector1 = vector1 - origin
        relative_vector2 = vector2 - origin

        angle_radians = np.arccos(np.dot(relative_vector1, relative_vector2) / (np.linalg.norm(relative_vector1) * np.linalg.norm(relative_vector2)))

        angle_degrees = np.degrees(angle_radians)
        return angle_degrees

    
    for i in range(0,len(mc)):
        left_foot_ankle_knee.append(find_angle(left_foot_index[i],left_ankle[i],left_knee[i]))
        right_foot_ankle_knee.append(find_angle(right_foot_index[i],right_ankle[i],right_knee[i]))

        right_ankle_knee_hip.append(find_angle(right_ankle[i],right_knee[i],right_hip[i]))
        left_ankle_knee_hip.append(find_angle(left_ankle[i],left_knee[i],left_hip[i]))
        
        right_knee_hip_shoulder.append(find_angle(right_knee[i],right_hip[i],right_shoulder[i]))
        left_knee_hip_shoulder.append(find_angle(left_knee[i],left_hip[i],left_shoulder[i]))

        right_hip_shoulder_elbow.append(find_angle(right_hip[i],right_shoulder[i],right_elbow[i]))
        left_hip_shoulder_elbow.append(find_angle(left_hip[i],left_shoulder[i],left_elbow[i]))

        right_shoulder_elbow_wrist.append(find_angle(right_shoulder[i],right_elbow[i],right_wrist[i]))
        left_shoulder_elbow_wrist.append(find_angle(left_shoulder[i],left_elbow[i],left_wrist[i]))

        



    seg_df["Time"]=mc["Time"]
    seg_df['left_foot_ankle_knee']=left_foot_ankle_knee
    seg_df['right_foot_ankle_knee']=right_foot_ankle_knee

    seg_df['right_ankle_knee_hip']=right_ankle_knee_hip
    seg_df['left_ankle_knee_hip']=left_ankle_knee_hip

    seg_df['right_knee_hip_shoulder']=right_knee_hip_shoulder
    seg_df['left_knee_hip_shoulder']=left_knee_hip_shoulder

    seg_df['right_hip_shoulder_elbow']=right_hip_shoulder_elbow
    seg_df['left_hip_shoulder_elbow']=left_hip_shoulder_elbow

    seg_df['right_shoulder_elbow_wrist']=right_shoulder_elbow_wrist
    seg_df['left_shoulder_elbow_wrist']=left_shoulder_elbow_wrist

    path="D:\ML\FYP\Dataset\\New_Dataset\\New\\"+sub+"\Outputs\\"
    seg_df.to_csv(path+activity+"_Angles_MC_T"+trial+".csv")



In [17]:

def cv_angles(sub,activity,cam,trial):

    df1=pd.read_csv("D:\ML\FYP\Dataset\\New_Dataset\\New\\"+sub+"\Outputs\\xyz\\"+sub+"_Cam"+cam+"_T"+trial+"_xyz.csv")
    
    cv_columns=["Time","Frame_Num","ON_LED","LEFT_ANKLE_X","LEFT_ANKLE_Y","LEFT_ANKLE_Z","LEFT_HEEL_X","LEFT_HEEL_Y","LEFT_HEEL_Z","LEFT_KNEE_X",	
    "LEFT_KNEE_Y",	"LEFT_KNEE_Z",	"LEFT_ELBOW_X",	"LEFT_ELBOW_Y",	"LEFT_ELBOW_Z",	"LEFT_SHOULDER_X",	"LEFT_SHOULDER_Y",	
    "LEFT_SHOULDER_Z","LEFT_FOOT_INDEX_X","LEFT_FOOT_INDEX_Y","LEFT_FOOT_INDEX_Z","LEFT_HIP_X","LEFT_HIP_Y",	
    "LEFT_HIP_Z","LEFT_WRIST_X","LEFT_WRIST_Y","LEFT_WRIST_Z","RIGHT_ANKLE_X","RIGHT_ANKLE_Y","RIGHT_ANKLE_Z","RIGHT_HEEL_X",	"RIGHT_HEEL_Y",	"RIGHT_HEEL_Z",	
    "RIGHT_KNEE_X",	"RIGHT_KNEE_Y",	"RIGHT_KNEE_Z",	"RIGHT_ELBOW_X","RIGHT_ELBOW_Y","RIGHT_ELBOW_Z","RIGHT_SHOULDER_X",	
    "RIGHT_SHOULDER_Y",	"RIGHT_SHOULDER_Z",	"RIGHT_FOOT_INDEX_X","RIGHT_FOOT_INDEX_Y","RIGHT_FOOT_INDEX_Z","RIGHT_HIP_X",	
    "RIGHT_HIP_Y",	"RIGHT_HIP_Z","RIGHT_WRIST_X","RIGHT_WRIST_Y","RIGHT_WRIST_Z"]
    
    cv=pd.DataFrame(df1, columns=cv_columns)
    
    temp=cv.iloc[:,0:3]
    cv=cv.drop(['Time','Frame_Num','ON_LED'],axis=1)
    cv.replace(0, np.nan, inplace=True)
    cv.apply(lambda x: x.fillna(x.mean()),axis=0)
    cv=pd.concat([temp,cv], axis=1, join='inner')
   
    cv.fillna(cv.mean(),inplace=True)

    diff_df=pd.DataFrame()

    left_ankle= cv[['LEFT_ANKLE_X', 'LEFT_ANKLE_Y','LEFT_ANKLE_Z']].values.tolist()
    left_heel=cv[['LEFT_HEEL_X', 'LEFT_HEEL_Y','LEFT_HEEL_Z']].values.tolist()
    left_foot_index=cv[['LEFT_FOOT_INDEX_X', 'LEFT_FOOT_INDEX_Y','LEFT_FOOT_INDEX_Z']].values.tolist()
    left_knee=cv[['LEFT_KNEE_X', 'LEFT_KNEE_Y','LEFT_KNEE_Z']].values.tolist()
    left_hip=cv[['LEFT_HIP_X', 'LEFT_HIP_Y','LEFT_HIP_Z']].values.tolist()
    left_shoulder=cv[['LEFT_SHOULDER_X', 'LEFT_SHOULDER_Y','LEFT_SHOULDER_Z']].values.tolist()
    left_elbow=cv[['LEFT_ELBOW_X', 'LEFT_ELBOW_Y','LEFT_ELBOW_Z']].values.tolist()
    left_wrist=cv[['LEFT_WRIST_X', 'LEFT_WRIST_Y','LEFT_WRIST_Z']].values.tolist()
    right_ankle= cv[['RIGHT_ANKLE_X', 'RIGHT_ANKLE_Y','RIGHT_ANKLE_Z']].values.tolist()
    right_heel=cv[['RIGHT_HEEL_X', 'RIGHT_HEEL_Y','RIGHT_HEEL_Z']].values.tolist()
    right_foot_index=cv[['RIGHT_FOOT_INDEX_X', 'RIGHT_FOOT_INDEX_Y','RIGHT_FOOT_INDEX_Z']].values.tolist()
    right_knee=cv[['RIGHT_KNEE_X', 'RIGHT_KNEE_Y','RIGHT_KNEE_Z']].values.tolist()
    right_hip=cv[['RIGHT_HIP_X', 'RIGHT_HIP_Y','RIGHT_HIP_Z']].values.tolist()
    right_shoulder=cv[['RIGHT_SHOULDER_X', 'RIGHT_SHOULDER_Y','RIGHT_SHOULDER_Z']].values.tolist()
    right_elbow=cv[['RIGHT_ELBOW_X', 'RIGHT_ELBOW_Y','RIGHT_ELBOW_Z']].values.tolist()
    right_wrist=cv[['RIGHT_WRIST_X', 'RIGHT_WRIST_Y','RIGHT_WRIST_Z']].values.tolist()
        
    seg_df=pd.DataFrame()
    
    
    left_foot_ankle_knee,right_foot_ankle_knee,right_ankle_knee_hip,left_ankle_knee_hip,right_knee_hip_shoulder,left_knee_hip_shoulder,right_hip_shoulder_elbow,left_hip_shoulder_elbow,right_shoulder_elbow_wrist,left_shoulder_elbow_wrist= ([] for i in range(10))
    
    def find_angle(l1,l2,l3):
        origin = np.array(l2)

        vector1 = np.array(l1)
        vector2 = np.array(l3)

        relative_vector1 = vector1 - origin
        relative_vector2 = vector2 - origin

        angle_radians = np.arccos(np.dot(relative_vector1, relative_vector2) / (np.linalg.norm(relative_vector1) * np.linalg.norm(relative_vector2)))

        angle_degrees = np.degrees(angle_radians)
        return angle_degrees

    
    for i in range(0,len(cv)):
        left_foot_ankle_knee.append(find_angle(left_foot_index[i],left_ankle[i],left_knee[i]))
        right_foot_ankle_knee.append(find_angle(right_foot_index[i],right_ankle[i],right_knee[i]))

        right_ankle_knee_hip.append(find_angle(right_ankle[i],right_knee[i],right_hip[i]))
        left_ankle_knee_hip.append(find_angle(left_ankle[i],left_knee[i],left_hip[i]))
        
        right_knee_hip_shoulder.append(find_angle(right_knee[i],right_hip[i],right_shoulder[i]))
        left_knee_hip_shoulder.append(find_angle(left_knee[i],left_hip[i],left_shoulder[i]))

        right_hip_shoulder_elbow.append(find_angle(right_hip[i],right_shoulder[i],right_elbow[i]))
        left_hip_shoulder_elbow.append(find_angle(left_hip[i],left_shoulder[i],left_elbow[i]))

        right_shoulder_elbow_wrist.append(find_angle(right_shoulder[i],right_elbow[i],right_wrist[i]))
        left_shoulder_elbow_wrist.append(find_angle(left_shoulder[i],left_elbow[i],left_wrist[i]))

        

    seg_df["Time"]=cv["Time"]
    seg_df['left_foot_ankle_knee']=left_foot_ankle_knee
    seg_df['right_foot_ankle_knee']=right_foot_ankle_knee

    seg_df['right_ankle_knee_hip']=right_ankle_knee_hip
    seg_df['left_ankle_knee_hip']=left_ankle_knee_hip

    seg_df['right_knee_hip_shoulder']=right_knee_hip_shoulder
    seg_df['left_knee_hip_shoulder']=left_knee_hip_shoulder

    seg_df['right_hip_shoulder_elbow']=right_hip_shoulder_elbow
    seg_df['left_hip_shoulder_elbow']=left_hip_shoulder_elbow

    seg_df['right_shoulder_elbow_wrist']=right_shoulder_elbow_wrist
    seg_df['left_shoulder_elbow_wrist']=left_shoulder_elbow_wrist

    path="D:\ML\FYP\Dataset\\New_Dataset\\New\\"+sub+"\Outputs\\"

    seg_df.to_csv(path+activity+"_Angles_Cam_"+cam+"_T"+trial+".csv")


In [18]:
def plot(sub,activity,cam,trial):  

    joints=['left_foot_ankle_knee','right_foot_ankle_knee','right_ankle_knee_hip','left_ankle_knee_hip','right_knee_hip_shoulder','left_knee_hip_shoulder','right_hip_shoulder_elbow','left_hip_shoulder_elbow','right_shoulder_elbow_wrist','left_shoulder_elbow_wrist']
    
    path="D:\ML\FYP\Dataset\\New_Dataset\\New\\"+sub+"\Outputs\\"
    os.chdir(path)
    with PdfPages(activity+'_NewvsMC_Cam'+cam+'_T'+trial+'.pdf') as pdf:
        for joint in joints:
            csv_file_path = path+activity+"_Angles_Cam_"+cam+"_T"+trial+".csv"
            

            plt.figure(figsize=(15, 9))

            plt.xlabel('Time')
            plt.ylabel(joint)
            plt.title('Time vs.' +joint)

            df = pd.read_csv(csv_file_path)

            time_data = df['Time']

            column = df[joint]

            plt.plot( time_data,column,label='New')

            csv_file_path = path+activity+"_Angles_MC_T"+trial+".csv"

            df = pd.read_csv(csv_file_path)

            time_data = df['Time']

            column = df[joint]
            plt.plot( time_data,column,label='MC')
            plt.legend()
            pdf.savefig()
            plt.show()



In [19]:
def all(sub,activity,cam, trial):
    count=video_to_frames(sub,activity,cam,trial)
    xyz(sub,activity,count,cam,trial)
    #clearing the frames from the folder
    os.chdir("D:\ML\\FYP\Dataset\\New_Dataset\\New\\"+sub+"\\Outputs\\frames")
    jpg_files = glob.glob('*.jpg')
    for jpg_file in jpg_files:
        os.remove(jpg_file)
    mc_angles(sub,activity,trial)
    cv_angles(sub,activity,cam,trial)
    plot(sub,activity,cam,trial)
    

In [ ]:

activities=['Trikonasana',"Dusting",'Malactivity','ClimbOnStool','Vrikshasana']
subjects=['S7','S8','S10']
trial=[1,3,5]
for a in activities:
    for s in subjects:
        for t in trial:
            all(sub=s,activity=a,cam='2',trial=str(t))